<h1>AHP</h1>


In [2]:
# Filename: AHP.py
# Description: Analytic hierarchy process method
# Authors: Papathanasiou, J. & Ploskas, N.

import numpy as np
import scipy.sparse.linalg as sc
import linalg  # Assuming standard numpy linalg or similar context
from numpy import array, sum, round, pow, real, vstack, transpose, amax
import matplotlib.pyplot as plt

# Note: This requires the local module AHP_Final_Rank_Figure to exist
try:
    from AHP_Final_Rank_Figure import graph, plot
except ImportError:
    def graph(x, y): print("Graphing: ", x)
    def plot(x, y): print("Plotting: ", x)

def norm(x):
    """
    Normalized column sum method.
    x is the pairwise comparison matrix for the criteria or the alternatives.
    """
    col_sum = array(sum(x, 0))
    z = array([[round(x[i, j] / col_sum[j], 3)
                for j in range(x.shape[1])]
                for i in range(x.shape[0])])
    return z

def geomean(x):
    """
    Geometric mean method.
    x is the pairwise comparison matrix for the criteria or the alternatives.
    """
    n = x.shape[0]
    z = [1.0] * n
    for i in range(n):
        for j in range(x.shape[1]):
            z[i] = z[i] * x[i][j]
        z[i] = pow(z[i], (1 / n))
    return z

def ahp(PCM, PCcriteria, m, n, c):
    """
    AHP method: coordinates priority vector estimation.
    PCM: pairwise comparison matrix for alternatives
    PCcriteria: pairwise comparison matrix for criteria
    m: number of alternatives
    n: number of criteria
    c: method flag (1: eigenvector, 2: normalized column sum, 3: geometric mean)
    """
    # 1. Calculate the priority vector of criteria (w)
    if c == 1:  # Eigenvector method
        val, vec = sc.eigs(PCcriteria, k=1, which='LM')
        eigcriteria = real(vec)
        w = eigcriteria / sum(eigcriteria)
        w = array(w).ravel()
    elif c == 2:  # Normalized column sum
        normPCcriteria = norm(PCcriteria)
        w = array(sum(normPCcriteria, 1) / n)
    else:  # Geometric mean
        GMcriteria = geomean(PCcriteria)
        w = GMcriteria / sum(GMcriteria)

    # 2. Calculate the local priority vectors for the alternatives (S)
    S = []
    for i in range(n):
        # Slice the PCM for the specific criterion
        subset_PCM = PCM[i * m : i * m + m, 0 : m]
        
        if c == 1:
            val, vec = sc.eigs(subset_PCM, k=1, which='LM')
            eigalter = real(vec)
            s = eigalter / sum(eigalter)
            s = array(s).ravel()
        elif c == 2:
            normPCM = norm(subset_PCM)
            s = array(sum(normPCM, 1) / m)
        else:
            GMalternatives = geomean(subset_PCM)
            s = GMalternatives / sum(GMalternatives)
        S.append(s)

    S = transpose(S)

    # 3. Calculate the global priority vector (v)
    v = S.dot(w.T)
    return v

def main(print_graph, print_plot, method_flag):
    """
    Main execution function.
    print_graph/print_plot: 'y' to enable visualization.
    method_flag: 1 (Eigen), 2 (Norm), 3 (Geo).
    """
    m = 6  # Alternatives
    n = 4  # Criteria
    
    # Random Indices for consistency checking
    RI = [0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49]

    # Pairwise comparison matrix of the criteria
    PCcriteria = array([
        [1, 1, 5, 3], 
        [1, 1, 5, 3],
        [1/5, 1/5, 1, 1/3], 
        [1/3, 1/3, 3, 1]
    ])

    # Consistency check for criteria
    lambdamax = amax(np.linalg.eigvals(PCcriteria).real)
    CI = (lambdamax - n) / (n - 1)
    CR = CI / RI[n - 1]
    print(f"Inconsistency index of the criteria: {CR:.4f}")
    if CR > 0.1:
        print("The pairwise comparison matrix of the criteria is inconsistent")

    # Pairwise comparison matrices for alternatives per criterion
    PCM1 = array([[1, 5, 1, 1, 1/3, 3], [1/5, 1, 1/3, 1/5, 1/7, 1], [1, 3, 1, 1/3, 1/5, 1], 
                  [1, 5, 3, 1, 1/3, 3], [3, 7, 5, 3, 1, 7], [1/3, 1, 1, 1/3, 1/7, 1]])
    
    PCM2 = array([[1, 7, 3, 1/3, 1/3, 1/3], [1/7, 1, 1/3, 1/7, 1/9, 1/7], [1/3, 3, 1, 1/5, 1/5, 1/5],
                  [3, 7, 5, 1, 1, 1], [3, 9, 5, 1, 1, 1], [3, 7, 5, 1, 1, 1]])
    
    PCM3 = array([[1, 1/9, 1/7, 1/9, 1, 1/5], [9, 1, 1, 1, 5, 3], [7, 1, 1, 1, 5, 1],
                  [9, 1, 1, 1, 7, 3], [1, 1/5, 1/5, 1/7, 1, 1/3], [5, 1/3, 1, 1/3, 3, 1]])
    
    PCM4 = array([[1, 1/5, 1/5, 1/3, 1/7, 1/5], [5, 1, 1, 3, 1/3, 1], [5, 1, 1, 1, 1/3, 1],
                  [3, 1/3, 1, 1, 1/7, 1], [7, 3, 3, 7, 1, 5], [5, 1, 1, 1, 1/5, 1]])

    allPCM = vstack((PCM1, PCM2, PCM3, PCM4))

    # Consistency check for alternatives
    for i in range(n):
        subset = allPCM[i * m : i * m + m, 0 : m]
        lambdamax = amax(np.linalg.eigvals(subset).real)
        CI = (lambdamax - m) / (m - 1)
        CR = CI / RI[m - 1]
        print(f"Inconsistency index for criterion {i+1}: {CR:.4f}")
        if CR > 0.1:
            print(f"The pairwise comparison matrix for criterion {i+1} is inconsistent")

    # Calculate results
    scores = ahp(allPCM, PCcriteria, m, n, method_flag)
    print("Global priorities =", scores)

    # Visualization
    if print_graph == 'y':
        graph(np.around(scores, 3), "Score")
    if print_plot == 'y':
        plot(np.around(scores, 3), "AHP")

if __name__ == '__main__':
    # Running with: No graph, Plot enabled, Method 1 (Eigenvector)
    main('n', 'y', 1)

Inconsistency index of the criteria: 0.0161
Inconsistency index for criterion 1: 0.0335
Inconsistency index for criterion 2: 0.0264
Inconsistency index for criterion 3: 0.0221
Inconsistency index for criterion 4: 0.0398
Global priorities = [0.11737682 0.07132502 0.09472062 0.21164765 0.35008026 0.15484963]
Plotting:  [0.117 0.071 0.095 0.212 0.35  0.155]
